<a href="https://colab.research.google.com/github/OSS-GeoAIDev/RAG/blob/main/RAG_App_using_Langchain_OpenAI_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai tiktoken rapidocr-onnxruntime

In [17]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [18]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!pip install langchain_community

In [1]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [2]:
url ="https://raw.githubusercontent.com/OSS-GeoAIDev/RAG/refs/heads/main/state_of_the_union.txt"

In [3]:
response = requests.get(url)

In [ ]:
print(response.text)

In [5]:
rawdata = response.text

In [6]:
with open("state_of_the_union.txt", "w") as f:
    f.write(rawdata)

In [7]:
loader = TextLoader('/content/state_of_the_union.txt')

In [8]:
document=loader.load()

In [ ]:
print(document[0].page_content)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [12]:
text_chunks=text_splitter.split_documents(document)

In [13]:
print(text_chunks[3].page_content)

The speech is generally held in January or February, and an invitation to the president is extended to use the chamber of the House by the speaker of the House. Starting in 1981, Ronald Reagan, the 40th U.S. president, began the practice of newly inaugurated presidents delivering an address to Congress in the first year of their term but not designating that speech an official "State of the Union".[7]


In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [19]:
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/tmp/ipython-input-3715417811.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [20]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.6 MB/s eta 0:00:00


In [21]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [22]:
retriever=vectorstore.as_retriever()

In [23]:
from langchain.prompts import ChatPromptTemplate

In [24]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [25]:
prompt=ChatPromptTemplate.from_template(template)

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [27]:
output_parser=StrOutputParser()

In [31]:
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-4o-mini")

In [32]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [34]:
rag_chain.invoke("Explain how speech applied in today's world?")

'Speech plays a crucial role in today\'s world as a primary means of communication, particularly in political contexts. The State of the Union address, for example, has evolved from a message between the president and Congress to a direct communication with the American people. With advancements in technology, such as radio and television, these speeches are broadcast live across various platforms to reach a wider audience. The timing of the speech is strategically chosen, typically at 9 p.m. Eastern Time, to maximize viewership. Additionally, the integration of the internet has allowed for real-time streaming, broadening accessibility beyond traditional media. Historically, the speech has undergone changes, such as the removal of time limits for coverage and the introduction of extensive commentary from political figures. The phrase "State of the Union" itself has become a recognized term since its first use by Franklin D. Roosevelt in 1934. Overall, speech remains a vital tool for sh